In [15]:
import pandas as pd
#from my_leagues import *
from functions.sleeper_functions import *
import requests
from datetime import datetime
import re

In [16]:
el = get_existing_leagues()

In [17]:
df = el.reset_index(drop=True)
df = df[df['status'] == 'in_season']
#df = df.dropna(subset=['previous_league_id'])
#df = df[df['previous_league_id'].str.len() > 2].reset_index(drop=True)

In [18]:
df = df.sort_values(by='last_message_time', ascending = False)

In [19]:
len(df)

56313

In [8]:
leagues = df.league_id.to_list()

In [21]:
tdf = pd.DataFrame()
start = datetime.now()
limit = 5000
ls = 0
for league in leagues[:limit]:
    ls += 1
    if ls % 500 == 0:
        pct = (ls/limit)
        span = (datetime.now() - start) / pct
        print(str(ls) + '/' + str(limit) + ' (' + "%.2f" % (pct * 100) + '%) ETR: ' + str(start + span))
    try:
        teams = get_league(league_id = league)#['metadata']

        for index, row in teams.iterrows():
            try:
                #print(row['user_id'], row['metadata'].get('team_name'))
                l = row['league_id']
                uid = row['user_id']
                tn = row['metadata'].get('team_name')
                rows = [l, uid, tn]
                tdf = tdf.append([rows])
            except Exception as ex:
                print(ex)
                pass
    except Exception as e:
        print(e)
        pass

print('Finished in: ' + str(datetime.now()-start))

500/5000 (10.00%) ETR: 2022-08-17 12:09:04.057812
1000/5000 (20.00%) ETR: 2022-08-17 12:09:01.676532
1500/5000 (30.00%) ETR: 2022-08-17 12:06:11.547249
2000/5000 (40.00%) ETR: 2022-08-17 12:06:22.443009
2500/5000 (50.00%) ETR: 2022-08-17 12:05:42.776076
3000/5000 (60.00%) ETR: 2022-08-17 12:05:07.427447
3500/5000 (70.00%) ETR: 2022-08-17 12:04:51.385372
4000/5000 (80.00%) ETR: 2022-08-17 12:04:29.663634
4500/5000 (90.00%) ETR: 2022-08-17 12:04:21.162666
5000/5000 (100.00%) ETR: 2022-08-17 12:04:42.254483
Finished in: 0:26:48.119935


In [22]:
tdf.columns = ['league', 'user_id','team_name']
tdf = tdf.dropna()
tdf['team_name'] = tdf.team_name.str.replace('&','and',regex=False).str.replace('[^a-zA-Z0-9 \n\.]', '', regex=True).str.lower().str.strip()
tdf['team_name'] = tdf.team_name.str.replace('^the ','',regex=True)
tdf = tdf.drop_duplicates(subset=['user_id','team_name']).reset_index(drop=True)
tdf.to_csv('Files/team_names.csv', index=False)

In [23]:
fin = tdf
fin = fin.groupby('team_name').count().sort_values(by='user_id', ascending = False).reset_index()

In [24]:
fin.to_csv('Files/top_team_names.csv', index=False)